Neural network takes data in csv form, as pair rgb, depth per line.

So we prepare data in required format and csv for them.

### converting files from tiff to jpg (RGB) and png (depth) and moving them to new folder

In [14]:
import glob
import os
from configparser import ConfigParser
import progressbar
from joblib import Parallel, delayed
from PIL import Image
import numpy as np
import csv
from visualization import load_depth

In [15]:
def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def convert_tiff(in_directory, out_directory, out_name, name, out_format, is_depth=False):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        # print(infile)
        if is_depth:
            depth = load_depth(name)
        else:
            im = Image.open(infile)
            im = im.convert(mode="RGB")
        # print("Generating jpeg for {}".format(name))
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def tiff_to_dataset(in_directory, out_directory, name, rgb_format, depth_format):
    convert_tiff(in_directory, out_directory, get_base_name(name), get_base_name(name)+'.tiff', rgb_format, is_depth=False)
    convert_tiff(in_directory, out_directory, get_base_name(name), get_base_name(name)+'-depth.tiff', depth_format, is_depth=True)


def ndc_to_real(in_directory, out_directory, name):
    # converts ndc depth to real depth in meters
    convert_tiff(in_directory, out_directory, get_base_name(name), get_base_name(name)+'.tiff', rgb_format)
    convert_tiff(in_directory, out_directory, get_base_name(name), get_base_name(name)+'-depth.tiff', depth_format)



In [16]:
ini_file = "gta-postprocessing.local.ini"

in_directory = 'D:\projekty\GTA-V-extractors\output'
out_directory = 'D:\projekty\GTA-V-extractors\depthEstimationNN1\data-gta'
out_root_directory = 'D:\projekty\GTA-V-extractors\depthEstimationNN1'

rgb_format = 'jpg'
depth_format = 'png'

import visualization
visualization.multi_page = False
visualization.ini_file = 'gta-postprocessing.local.ini'

# CONFIG = ConfigParser()
# CONFIG.read(ini_file)
# in_directory = CONFIG["Images"]["Tiff"]
# out_directory = CONFIG["Images"]["Jpg"]

In [ ]:
workers = 10
pattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]--[0-9][0-9]-[0-9][0-9]-[0-9][0-9]--[0-9][0-9][0-9].tiff'

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

files = glob.glob(os.path.join(in_directory, pattern))

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(files) * 2).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(tiff_to_dataset)(in_directory, out_directory, name, rgb_format, depth_format)
                         for name in files)


In [12]:
files = glob.glob(os.path.join(in_directory, pattern))

filenames = []
for i, name in enumerate(files):
    rgb_name = "{}.{}".format(get_base_name(name), rgb_format)
    rgb_name = os.path.relpath(os.path.join(out_directory, rgb_name), start=out_root_directory)
    rgb_name = rgb_name.replace('\\', '/')
    depth_name = "{}.{}".format(get_base_name(name), depth_format)
    depth_name = os.path.relpath(os.path.join(out_directory, depth_name), start=out_root_directory)
    depth_name = depth_name.replace('\\', '/')
    filenames.append([rgb_name, depth_name]) 

    # r je abych nemusel psát zpětná lomítka
with open(os.path.join(out_root_directory, 'train-gta.csv'), 'w+', newline='') as f:
    csv.writer(f).writerows(filenames)
